# **Mengimport Libraries**

Mengimport libraries yang diperlukan seperti torch, torchvision pil, dll

In [1]:
import torch
from torchvision.models import resnet50, ResNet50_Weights, mobilenet_v3_large, MobileNet_V3_Large_Weights, mobilenet_v2, MobileNet_V2_Weights
from torch import nn
from torch import optim
from PIL import Image
import os
import sys
import re
import tqdm
import random 
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.optim import lr_scheduler
import cv2
import imageio
from facenet_pytorch import MTCNN
from matplotlib.pyplot import imshow
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import cv2
import torch
import numpy as np
from PIL import Image, ImageOps
from tqdm import tqdm
from facenet_pytorch import MTCNN
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor

d:\anaconda3\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Preprocessing**

Membuat kelas preprocessing menggunakan MTCNN yang dimana juga ditambahkan rotasi gambar biar wajahnya sejajar dan membuat subfolder untuk menyimpan gambar setiap kelas

In [ ]:
import os
import re
import gc
import math
import torch
from PIL import Image, ImageOps
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from ultralytics import YOLO
from facenet_pytorch import MTCNN

class FaceCropper:
    def __init__(self, mtcnn_device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.mtcnn = MTCNN(keep_all=False, device=mtcnn_device)
        self.yolo_fallback = YOLO(
            r"D:\Models\huggingface\hub\models--arnabdhar--YOLOv8-Face-Detection\snapshots\52fa54977207fa4f021de949b515fb19dcab4488\model.pt"
        )

    def extract_class(self, txt):
        raw_class = re.split(r"[-_.]", txt, 2)[0:2]
        complete_class = raw_class[0] + "_" + raw_class[1]
        complete_class = re.sub(r'[\d\s]+$', '', complete_class)
        return complete_class
    
    def auto_rotate_image(self, image):
        """Rotate image using MTCNN landmarks"""
        _, _, landmarks = self.mtcnn.detect(image, landmarks=True)
        if landmarks is not None:
            left_eye, right_eye = landmarks[0][:2]
            dx, dy = right_eye[0] - left_eye[0], right_eye[1] - left_eye[1]
            angle = math.degrees(math.atan2(dy, dx))
            image = image.rotate(-angle, expand=True)
        return image
    
    def crop_face(self, image_path):
        try:
            image = Image.open(image_path)
            image = ImageOps.exif_transpose(image)
            image = self.auto_rotate_image(image)
    
            # First try MTCNN
            boxes, _ = self.mtcnn.detect(image)
            if boxes is not None and len(boxes) > 0:
                box = boxes[0]
            else:
                # Fall back to YOLO
                results = self.yolo_fallback(image_path)
                if results and len(results[0].boxes.xyxy) > 0:
                    box = results[0].boxes.xyxy[0].cpu().numpy()
                else:
                    return None
    
            left, top, right, bottom = map(int, box)
            return image.crop((left, top, right, bottom))
        except Exception as e:
            print(f"Failed to process {image_path}: {e}")
            return None
    
    def process_folder(self, input_dir, output_dir, batch_size=32, max_workers=8):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i + batch_size]
        
            def process_file(fname):
                path = os.path.join(input_dir, fname)
                cropped = self.crop_face(path)
                if cropped:
                    class_name = self.extract_class(fname)
                    class_dir = os.path.join(output_dir, class_name)
                    os.makedirs(class_dir, exist_ok=True)  # Create class subfolder safely
                    save_path = os.path.join(class_dir, fname)  # keep original filename
                    # If file exists, append counter
                    counter = 1
                    base, ext = os.path.splitext(save_path)
                    while os.path.exists(save_path):
                        save_path = f"{base}_{counter}{ext}"
                        counter += 1
                    cropped.save(save_path)
        
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                list(tqdm(executor.map(process_file, batch_files), total=len(batch_files)))
        
            # Clean memory
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

Menggunakan kelas preprocessing yang sudah dibuat

In [ ]:
# Inisiasi cropper
cropper = FaceCropper()

# --- Train paths ---
train_input = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train"
train_output = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"

print("🟢 Starting TRAIN image cropping...")
cropper.process_folder(train_input, train_output)

# --- Test paths ---
test_input = train_input.replace("Train", "Test")
test_output = train_output.replace("Train", "Test")

print("\n🟣 Starting TEST image cropping...")
cropper.process_folder(test_input, test_output)

print("\n✅ All done! Cropped images saved successfully.")

d:\anaconda3\envs\deeplearning\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
d

🟢 Starting TRAIN image cropping...


 47%|████▋     | 15/32 [00:11<00:16,  1.06it/s]

# **Membuat kelas ArcFaceLoss sebagai loss dari model nanti**

ArcFaceLoss dipilih karena untuk face recognition arcfaceloss yang memberikan generelesasi paling baik dan tidak overfitting selain itu convergenya cepet tidak seperti Triplet loss

In [3]:
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return nn.CrossEntropyLoss()(logits, labels)
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)

# **Membuat MobilenetEmbedding**

Membuat embedding face recognition menggunakan mobilenet, dimana hasil terbaik ternyata waktu weightnya tidak di freeze

In [4]:
class MobilenetEmbedding(nn.Module):
    def __init__(self, embedding_size=128, weights=None):
        super().__init__()
        # self.mobilenet = mobilenet_v3_large(weights=weights)
        self.mobilenet = mobilenet_v2(weights=weights) 
        self.mobilenet.classifier[1] = nn.Linear(1280, embedding_size)
        # if weights is not None:
        #     for param in self.mobilenet.parameters():
        #         param.requires_grad = False
        #     for param in self.mobilenet.classifier[1].parameters():
        #         param.requires_grad = True

    def forward(self, x):
        x = self.mobilenet(x)
        x = F.normalize(x, p=2, dim=1)
        return x

## **Training model mobilenetv2_arcface_best**

Kita memasukan dataset ke class datasets menggunakan transformasi yang dimana trainingnya itu ditambahkan beberapa data augmentasi karena sebelum ini masih overfit dan untuk realtime tidak terlalu bagus lalu dimasukan ke-dataloader dan training dimulai. Disimpan model weightnya saja

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobilenetEmbedding(
    embedding_size=128,
    weights=MobileNet_V2_Weights.IMAGENET1K_V1
).to(device)

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=128,
    margin=0.3,
    scale=30.0
).to(device)

optimizer = optim.Adam(model.parameters(), lr=1.5e-4, weight_decay=5e-5)

# --- Training Loop ---
num_epochs = 15
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None

for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

Epoch 1/15 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.38it/s]


Epoch 1 | Train Loss: 11.1741 | Test Loss: 9.3295


Epoch 2/15 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.36it/s]


Epoch 2 | Train Loss: 4.1952 | Test Loss: 4.6143


Epoch 3/15 - Validation: 100%|██████████| 27/27 [00:21<00:00,  1.28it/s]


...

Epoch 13 | Train Loss: 0.0307 | Test Loss: 0.7215


Epoch 14/15 - Validation: 100%|██████████| 27/27 [00:20<00:00,  1.34it/s]


Epoch 14 | Train Loss: 0.0381 | Test Loss: 0.7199


Epoch 15/15 - Validation: 100%|██████████| 27/27 [00:20<00:00,  1.33it/s]


Epoch 15 | Train Loss: 0.0538 | Test Loss: 0.7997
✅ Best model saved to: D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth


## **Training mobilenetv2_arcface_diffmargin_moreagument_best.pth**

Sama seperti sebelumnya tetapi margin dari arcfacelossnya ditambahkan 

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobilenetEmbedding(
    embedding_size=128,
    weights=MobileNet_V2_Weights.IMAGENET1K_V1
).to(device)

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=128,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=1.5e-4, weight_decay=5e-5)

# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None

for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

Epoch 1/50 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.39it/s]


Epoch 1 | Train Loss: 17.7598 | Test Loss: 16.6339


Epoch 2/50 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.39it/s]


Epoch 2 | Train Loss: 13.3814 | Test Loss: 13.4777


Epoch 3/50 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.41it/s]


...

Epoch 23 | Train Loss: 0.0875 | Test Loss: 1.2250


Epoch 24/50 - Validation: 100%|██████████| 27/27 [00:19<00:00,  1.36it/s]


Epoch 24 | Train Loss: 0.1289 | Test Loss: 1.3865


Epoch 25/50 - Validation: 100%|██████████| 27/27 [00:20<00:00,  1.31it/s]

Epoch 25 | Train Loss: 0.0814 | Test Loss: 1.2364
Early stopping triggered after 25 epochs.
✅ Best model saved to: D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best.pth


## **Training mobilenetv2_arcface_diffmargin_moreagument_best_0-2**

Marginya sama sebelumnya, tetapi transformationya ditambah karena masih overfit lossnya. Melatih 3 model

In [ ]:
## INI NYOBA MENAMBAH transformation
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.2), # 20% kemungkinan gambar akan diubah jadi grayscale
    transforms.ColorJitter(brightness=(0.2, 1.3),  # Kecerahan antara 40% - 100% (tidak pernah lebih terang)
                            contrast=(0.7, 1.3),  # Sedikit variasi kontras
                            saturation=0.2, 
                            hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobilenetEmbedding(
    embedding_size=128,
    weights=MobileNet_V2_Weights.IMAGENET1K_V2
).to(device)

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=128,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0005598380474533065, weight_decay=0.0005501701817196964)
# optimizer = torch.optim.SGD(model.parameters(), 
#                             lr=0.1, 
#                             momentum=0.9, 
#                             weight_decay=5e-4)
# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None
LR_MIN_EXP = -4  # 10**-4 = 0.0001
LR_MAX_EXP = -3  # 10**-3 = 0.001

# 2. Weight Decay (wd):
#    Biasanya antara 1e-5 dan 1e-3.
WD_MIN_EXP = -5  # 10**-5 = 0.00001
WD_MAX_EXP = -3  # 10**-3 = 0.001
# for i in range(20):
    # lr = np.random.uniform(10**LR_MIN_EXP, 10**LR_MAX_EXP)
    # wd = np.random.uniform(10**WD_MIN_EXP, 10**WD_MAX_EXP)
    # print(f"lr : {lr} | wd: {wd}")
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_2.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

Epoch 1/50 - Validation: 100%|██████████| 27/27 [00:21<00:00,  1.28it/s]


Epoch 1 | Train Loss: 18.0244 | Test Loss: 16.5904


Epoch 2/50 - Validation: 100%|██████████| 27/27 [00:22<00:00,  1.22it/s]


Epoch 2 | Train Loss: 12.8248 | Test Loss: 12.7710


Epoch 3/50 - Validation: 100%|██████████| 27/27 [00:21<00:00,  1.23it/s]


...

Epoch 17 | Train Loss: 0.2587 | Test Loss: 1.6808


Epoch 18/50 - Validation: 100%|██████████| 27/27 [00:20<00:00,  1.29it/s]


Epoch 18 | Train Loss: 0.1835 | Test Loss: 1.6515


Epoch 19/50 - Validation: 100%|██████████| 27/27 [00:21<00:00,  1.25it/s]

Epoch 19 | Train Loss: 0.2681 | Test Loss: 1.8248
Early stopping triggered after 19 epochs.
✅ Best model saved to: D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_2.pth


# **Evaluasi model embedding menggunakan cosine similarity**

Karena model bukan classifier langsung 54 kelas, jadi kita masih harus kalau ga pake model ml lagi menggunakan ml lain, atau kita tambahkan fully connected diakhir lalu embeddingnya kita freeze weightnya, atau kita menggunakan cosine similarity. Saya memilih cosine similarity

In [47]:
import torch
import numpy as np

train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Model ---
model = MobilenetEmbedding(embedding_size=128).to(device)
model_list = [r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth",
              r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best.pth", 
              r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_1.pth" , 
              r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_2.pth"]
for i in model_list:
    # model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth", weights_only=True))
    model.load_state_dict(torch.load(i, weights_only=True))
    # model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_2.pth", weights_only=True))

    model.eval()

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

    # --- Datasets ---
    train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

    # --- Dataloaders ---
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    # --- Extract train embeddings ---
    X_train = []
    y_train = []

    with torch.no_grad():
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            emb = model(imgs)
            X_train.append(emb.cpu().numpy())
            y_train.append(labels.numpy())

    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)

    # --- Extract test embeddings ---
    X_test = []
    y_test = []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            emb = model(imgs)
            X_test.append(emb.cpu().numpy())
            y_test.append(labels.numpy())

    X_test = np.vstack(X_test)
    y_test = np.hstack(y_test)

    # --- Build class prototypes ---
    classes = np.unique(y_train)
    # Build prototypes with class indices aligned to ImageFolder
    prototypes = {}
    for idx, cls in enumerate(train_dataset.classes):  # use ImageFolder.classes
        class_idx = train_dataset.class_to_idx[cls]
        prototypes[class_idx] = X_train[y_train == class_idx].mean(axis=0)
        prototypes[class_idx] /= np.linalg.norm(prototypes[class_idx])

    # --- Prediction function ---
    def predict_embedding(embedding, prototypes, threshold=0.5):
        embedding = embedding / np.linalg.norm(embedding)
        best_cls = None
        best_sim = -1
        for cls_idx, proto in prototypes.items():
            sim = np.dot(embedding, proto)
            if sim > best_sim:
                best_sim = sim
                best_cls = cls_idx
        if best_sim < threshold:
            return -1, best_sim  # Unknown
        return best_cls, best_sim

    # --- Predict on test set ---
    predictions = []
    similarities = []

    for emb in X_test:
        cls, sim = predict_embedding(emb, prototypes, threshold=0.5)
        predictions.append(cls)
        similarities.append(sim)

    predictions_test = np.array(predictions)
    similarities_test = np.array(similarities)
    
    # -- Prediction on training set --
    predictions = []
    similarities = []

    for emb in X_train:
        cls, sim = predict_embedding(emb, prototypes, threshold=0.5)
        predictions.append(cls)
        similarities.append(sim)

    predictions_train = np.array(predictions)
    similarities_train = np.array(similarities)

    # --- Accuracy ---
    acc_train = np.mean(predictions_train == y_train)
    acc_test = np.mean(predictions_test == y_test)
    filename = os.path.basename(i)
    print(f"✅ {filename} | Train Acc: {acc_train*100:.2f}% | Test Acc: {acc_test*100:.2f}%")



✅ mobilenetv2_arcface_best.pth | Train Acc: 100.00% | Test Acc: 96.64%
✅ mobilenetv2_arcface_diffmargin_moreagument_best.pth | Train Acc: 100.00% | Test Acc: 97.57%
✅ mobilenetv2_arcface_diffmargin_moreagument_best_1.pth | Train Acc: 100.00% | Test Acc: 97.57%
✅ mobilenetv2_arcface_diffmargin_moreagument_best_2.pth | Train Acc: 100.00% | Test Acc: 95.25%


# **Membuat model MobileFaceNet**

Model MobileFaceNet ini terinspirasi basenya jadi mobilenet sama2 ringan, hanya saja ini khusus untuk face recognition embedding menggunakan facearc loss, dan dia trainingnya menggunakan dataset wajah tidak seperti mobilenet yang imagenet dan tidak ada orang lain yang membagi bobot mobilenet ditrain untuk face recognition

In [10]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
from torch import nn
import torch
import math

class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

##################################  MobileFaceNet #############################################################
    
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)
class MobileFaceNet(Module):
    def __init__(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, 512, bias=False)
        self.bn = BatchNorm1d(512)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    def forward(self, x):
        # print("Input:", x.shape)
    
        out = self.conv1(x)
        # print("After conv1:", out.shape)
        
        out = self.conv2_dw(out)
        # print("After conv2_dw:", out.shape)

        out = self.conv_23(out)
        # print("After conv_23:", out.shape)

        out = self.conv_3(out)
        # print("After conv_3:", out.shape)
        
        out = self.conv_34(out)
        # print("After conv_34:", out.shape)

        out = self.conv_4(out)
        # print("After conv_4:", out.shape)

        out = self.conv_45(out)
        # print("After conv_45:", out.shape)

        out = self.conv_5(out)
        # print("After conv_5:", out.shape)

        out = self.conv_6_sep(out)
        # print("After conv_6_sep:", out.shape)

        out = self.conv_6_dw(out)
        # print("After conv_6_dw:", out.shape)

        out = self.conv_6_flatten(out)
        # print("After conv_6_flatten:", out.shape)

        out = self.linear(out)
        # print("After linear:", out.shape)

        out = self.bn(out)
        # print("After batchnorm:", out.shape)

        out = l2_norm(out)
        # print("After l2_norm:", out.shape)

        return out

## **Training MobileFaceNet**

Training modelnya setelah menemukan lr dan wd yang lumayan bagus dengan epoch sebanyaknya karena menggunakan early stopping. Selain itu saya menyimpan snapshot weight dari model dengan test loss terbaik. Untuk transformasinya kita tambahkan banyak data augmentasi agar tidak overfit, logika training nya sama seperti sebelumnya optimizer pakai adam.

In [ ]:
## INI NYOBA MENAMBAH PAKE LAIN
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.2), # 20% kemungkinan gambar akan diubah jadi grayscale
    transforms.ColorJitter(brightness=(0.2, 1.3),  # Kecerahan antara 40% - 100% (tidak pernah lebih terang)
                            contrast=(0.7, 1.3),  # Sedikit variasi kontras
                            saturation=0.2, 
                            hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\MobileFace_Net", map_location=lambda storage, loc: storage))

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=512,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0005598380474533065, weight_decay=0.0005501701817196964)
# optimizer = torch.optim.SGD(model.parameters(), 
#                             lr=0.1, 
#                             momentum=0.9, 
#                             weight_decay=5e-4)
# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None
LR_MIN_EXP = -4  # 10**-4 = 0.0001
LR_MAX_EXP = -3  # 10**-3 = 0.001

# 2. Weight Decay (wd):
#    Biasanya antara 1e-5 dan 1e-3.
WD_MIN_EXP = -5  # 10**-5 = 0.00001
WD_MAX_EXP = -3  # 10**-3 = 0.001
# for i in range(20):
    # lr = np.random.uniform(10**LR_MIN_EXP, 10**LR_MAX_EXP)
    # wd = np.random.uniform(10**WD_MIN_EXP, 10**WD_MAX_EXP)
    # print(f"lr : {lr} | wd: {wd}")
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_best.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

C:\Users\ramap\AppData\Local\Temp\ipykernel_13204\3093528768.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Lear

Epoch 1 | Train Loss: 16.0721 | Test Loss: 13.1650


Epoch 2/50 - Validation: 100%|██████████| 27/27 [00:25<00:00,  1.07it/s]


Epoch 2 | Train Loss: 9.0412 | Test Loss: 6.7330


Epoch 3/50 - Validation: 100%|██████████| 27/27 [00:25<00:00,  1.07it/s]


...

Epoch 13 | Train Loss: 0.2368 | Test Loss: 1.4209


Epoch 14/50 - Validation: 100%|██████████| 27/27 [00:33<00:00,  1.25s/it]


Epoch 14 | Train Loss: 0.1960 | Test Loss: 1.0150


Epoch 15/50 - Validation: 100%|██████████| 27/27 [00:25<00:00,  1.05it/s]


Epoch 15 | Train Loss: 0.2686 | Test Loss: 1.4817
Early stopping triggered after 15 epochs.
✅ Best model saved to: D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_best.pth


## **Hasil evaluasi MobileFaceNet**

Akurasinya bukan terbaik tetapi dari terbaik berbeda sedikit, dan ini lebih generelesasi kalau direaltime bener2 bagus dibanding mobilenetembedding. Strategi evaluasinya sama kaya mobilenetembedding

In [11]:
import torch
import numpy as np

train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Model ---
# model = MobilenetEmbedding(embedding_size=128).to(device)
# model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth", weights_only=True))
# model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best.pth", weights_only=True))
# model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_2.pth", weights_only=True))

model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_best.pth", map_location=lambda storage, loc: storage))
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
model.eval()

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Extract train embeddings ---
X_train = []
y_train = []

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        emb = model(imgs)
        X_train.append(emb.cpu().numpy())
        y_train.append(labels.numpy())

X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

# --- Extract test embeddings ---
X_test = []
y_test = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        emb = model(imgs)
        X_test.append(emb.cpu().numpy())
        y_test.append(labels.numpy())

X_test = np.vstack(X_test)
y_test = np.hstack(y_test)

# --- Build class prototypes ---
classes = np.unique(y_train)
# Build prototypes with class indices aligned to ImageFolder
prototypes = {}
for idx, cls in enumerate(train_dataset.classes):  # use ImageFolder.classes
    class_idx = train_dataset.class_to_idx[cls]
    prototypes[class_idx] = X_train[y_train == class_idx].mean(axis=0)
    prototypes[class_idx] /= np.linalg.norm(prototypes[class_idx])

# --- Prediction function ---
def predict_embedding(embedding, prototypes, threshold=0.5):
    embedding = embedding / np.linalg.norm(embedding)
    best_cls = None
    best_sim = -1
    for cls_idx, proto in prototypes.items():
        sim = np.dot(embedding, proto)
        if sim > best_sim:
            best_sim = sim
            best_cls = cls_idx
    if best_sim < threshold:
        return -1, best_sim  # Unknown
    return best_cls, best_sim

# --- Predict on test set ---
predictions = []
similarities = []

for emb in X_test:
    cls, sim = predict_embedding(emb, prototypes, threshold=0.5)
    predictions.append(cls)
    similarities.append(sim)

predictions_test = np.array(predictions)
similarities_test = np.array(similarities)

# -- Prediction on training set --
predictions = []
similarities = []

for emb in X_train:
    cls, sim = predict_embedding(emb, prototypes, threshold=0.5)
    predictions.append(cls)
    similarities.append(sim)

predictions_train = np.array(predictions)
similarities_train = np.array(similarities)

# --- Accuracy ---
acc_train = np.mean(predictions_train == y_train)
acc_test = np.mean(predictions_test == y_test)
filename = os.path.basename(i)
print(f"✅ {filename} | Train Acc: {acc_train*100:.2f}% | Test Acc: {acc_test*100:.2f}%")

C:\Users\ramap\AppData\Local\Temp\ipykernel_19404\3016627567.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Lear

✅ mobilenetv2_arcface_diffmargin_moreagument_best.pth | Train Acc: 100.00% | Test Acc: 97.22%


## **Contoh banyak wajah**

In [ ]:
class_names = {v: k for k, v in train_dataset.class_to_idx.items()}
normalize_transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


mtcnn = MTCNN(
    image_size=112, 
    margin=0, 
    keep_all=True, 
    post_process=False, # We will do our own normalization
    device=device
) 

# path = r"D:\download_d\IMG_1187.JPG"
# path = r"D:\download_d\IMG_1186.JPG"
# path = r"D:\download_d\IMG_1185.JPG"
# path = r"D:\download_d\IMG_1182.JPG"
# path = r"D:\download_d\IMG_1180.JPG"
# path = r"D:\download_d\IMG_1172.JPG"
path = r"D:\download_d\IMG_1158 (1).JPG"
# Load original image (for display only)
try:
    cv2_image = cv2.imread(path)
    if cv2_image is None:
        raise FileNotFoundError(f"Could not read image from path: {path}")
    original_for_display = cv2_image.copy()  # keep original copy
    
    # Load PIL image (copy for MTCNN) 
    pil_image_for_mtcnn = Image.open(path).convert('RGB')
except FileNotFoundError as e:
    print(e)
    sys.exit(0)
except Exception as e:
    print(f"An error occurred loading the image: {e}")
    sys.exit(0)


# --- Detect faces and get aligned tensors ---
# This call *already* uses landmarks internally for alignment/rotation
face_tensors = mtcnn(pil_image_for_mtcnn) 

if face_tensors is None:
    print("No faces detected.")
    sys.exit(0)

print(f"Detected {len(face_tensors)} faces.")

# --- Preprocess face tensors only ---
face_tensors = face_tensors.to(device) / 255.0
face_tensors = normalize_transform(face_tensors)

# --- Get embeddings ---
model.eval() # Set model to evaluation mode
with torch.no_grad():
    embeddings = model(face_tensors)

# --- Get boxes AND landmarks for drawing ---
# We run detect again, this time asking for landmarks to draw
boxes, probs, landmarks = mtcnn.detect(np.array(pil_image_for_mtcnn), landmarks=True) 

# --- Draw on original image ---
if boxes is not None:
    # Iterate over boxes and landmarks together
    for i, (box, lms) in enumerate(zip(boxes, landmarks)):
        emb = embeddings[i].cpu().numpy()
        cls_idx, sim = predict_embedding(emb, prototypes, threshold=0.5)
        
        name = "Unknown" if cls_idx == -1 else class_names[cls_idx]
        label = f"{name} ({sim*100:.1f}%)"
        
        x1, y1, x2, y2 = [int(b) for b in box]
        
        # Draw bounding box
        cv2.rectangle(original_for_display, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw label background
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(original_for_display, (x1, y1 - h - 10), (x1 + w, y1 - 5), (0, 255, 0), -1)
        
        # Draw label text
        cv2.putText(original_for_display, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        # *** NEW: Draw landmarks ***
        for point in lms:
            cv2.circle(original_for_display, (int(point[0]), int(point[1])), 2, (0, 0, 255), 2)

# --- Show the original image (no preprocessing applied) ---
window_name = 'Face Recognition'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
# cv2.resizeWindow(window_name, 800, 600) # You can adjust this as needed
cv2.imshow(window_name, original_for_display)
cv2.waitKey(0)
cv2.destroyAllWindows()

Detected 9 faces.


# **Realtime**

In [45]:
import torch
import numpy as np
import cv2
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from facenet_pytorch import MTCNN
from PIL import Image
import time



# --- Variabel & Path (dari skrip Anda) ---
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
model_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_1.pth"
# model_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best_1.pth"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

# --- Model ---
# model = MobilenetEmbedding(embedding_size=128).to(device)
# # model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_best.pth", weights_only=True))
# model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_moreagument_best.pth", weights_only=True))
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_best.pth", map_location=lambda storage, loc: storage))

model.eval()

# --- Transformasi ---
# Transformasi untuk model embedding
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ==============================================================================
# TAHAP 1: MEMBANGUN PROTOTIPE KELAS (Dijalankan sekali saat startup)
# ==============================================================================
print("Loading dataset pelatihan untuk membangun prototipe...")

# --- Datasets & Dataloaders (Hanya untuk train set) ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# Dapatkan nama kelas
class_names = train_dataset.classes
print(f"Ditemukan {len(class_names)} kelas: {class_names}")

# --- Ekstraksi embedding pelatihan ---
X_train = []
y_train = []

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        emb = model(imgs)
        X_train.append(emb.cpu().numpy())
        y_train.append(labels.numpy())

X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

print(f"Ekstraksi {len(X_train)} embedding pelatihan selesai.")

# --- Membangun prototipe kelas (dari skrip Anda) ---
prototypes = {}
for idx, cls in enumerate(train_dataset.classes):  # use ImageFolder.classes
    class_idx = train_dataset.class_to_idx[cls]
    class_embeddings = X_train[y_train == class_idx]
    
    if len(class_embeddings) > 0:
        prototypes[class_idx] = class_embeddings.mean(axis=0)
        prototypes[class_idx] /= np.linalg.norm(prototypes[class_idx])
    else:
        print(f"Peringatan: Tidak ada sampel ditemukan untuk kelas {cls} (idx: {class_idx})")

print(f"Prototipe untuk {len(prototypes)} kelas berhasil dibuat.")

# ==============================================================================
# TAHAP 2: FUNGSI HELPER
# ==============================================================================

# --- Fungsi Prediksi (dari skrip Anda) ---
def predict_embedding(embedding, prototypes, threshold=0.8):
    if embedding.ndim == 0 or np.linalg.norm(embedding) == 0:
        return -1, 0.0 # Embedding tidak valid
    
    embedding = embedding / np.linalg.norm(embedding)
    best_cls = None
    best_sim = -1
    
    for cls_idx, proto in prototypes.items():
        sim = np.dot(embedding, proto)
        if sim > best_sim:
            best_sim = sim
            best_cls = cls_idx
            
    if best_sim < threshold:
        return -1, best_sim  # Unknown
        
    return best_cls, best_sim

# --- Fungsi Pencerah Gambar ---
def auto_adjust_brightness(frame, target_brightness=90, low_threshold=70):
    """Mencerahkan frame jika brightness rata-rata di bawah ambang batas."""
    
    # Konversi ke Grayscale untuk mengecek brightness
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mean_brightness = np.mean(gray)
    
    if mean_brightness < low_threshold:
        # Hitung seberapa banyak brightness perlu ditambah
        delta = target_brightness - mean_brightness
        
        # Tambah brightness menggunakan cv2.convertScaleAbs untuk penanganan saturasi
        # alpha=1 (kontras tetap), beta=delta (brightness ditambah)
        bright_frame = cv2.convertScaleAbs(frame, alpha=1.0, beta=delta)
        return bright_frame
        
    return frame

# ==============================================================================
# TAHAP 3: REAL-TIME LOOP
# ==============================================================================

# --- Inisialisasi MTCNN ---
mtcnn = MTCNN(
    keep_all=True,              # Deteksi semua wajah
    post_process=False,         # Jangan lakukan normalisasi (kita lakukan di transform)
    min_face_size=40,           # Abaikan wajah yang sangat kecil
    device=device
)

# --- Inisialisasi Video Capture ---
# Ganti URL ini dengan URL dari aplikasi IP Webcam Anda
# Contoh: "http://192.168.1.10:8080/video"
# video_url = "http://YOUR_PHONE_IP:8080/video"

# Untuk testing menggunakan Webcam internal, ganti 'video_url' dengan 0
# cap = cv2.VideoCapture(0) 
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print(f"Error: Tidak bisa membuka stream video.")
    print(f"Pastikan URL '{video_url}' benar dan aplikasi IP Webcam berjalan.")
    exit()

print("Memulai stream video... Tekan 'q' untuk keluar.")

# Variabel untuk FPS
frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame, mencoba menghubungkan kembali...")
        cap.release()
        cap = cv2.VideoCapture(video_url)
        time.sleep(2)
        continue

    # 1. Pencerahan Otomatis (Sesuai permintaan Anda)
    frame = auto_adjust_brightness(frame)

    # 2. Konversi BGR (OpenCV) ke RGB (PIL) untuk MTCNN
    # MTCNN mengharapkan input PIL Image
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 3. Deteksi Wajah dengan MTCNN
    boxes, _, landmarks = mtcnn.detect(img_pil, landmarks=True)

    # 4. Loop untuk setiap wajah yang terdeteksi
    if boxes is not None:
        for i, box in enumerate(boxes):  # 'i' ditambahkan di sini
            x1, y1, x2, y2 = [int(b) for b in box]

            # Pastikan koordinat valid (tidak keluar dari frame)
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame.shape[1] - 1, x2)
            y2 = min(frame.shape[0] - 1, y2)

            # Pastikan crop valid
            if y2 <= y1 or x2 <= x1:
                continue

            # 5. Crop wajah dari frame BGR asli
            face_crop_bgr = frame[y1:y2, x1:x2]


            # 5.5 ALIGN Wajah menggunakan Landmark Mata
            try:
                # Ambil landmark untuk wajah ini
                current_landmarks = landmarks[i]
                left_eye = current_landmarks[0]   # [x, y] mata kiri
                right_eye = current_landmarks[1]  # [x, y] mata kanan
                
                # Hitung sudut antara dua mata
                delta_y = right_eye[1] - left_eye[1]
                delta_x = right_eye[0] - left_eye[0]
                # Hitung sudut dalam derajat
                angle_deg = np.degrees(np.arctan2(delta_y, delta_x))

                # Dapatkan center dari face crop (untuk titik rotasi)
                (h_crop, w_crop) = face_crop_bgr.shape[:2]
                center_crop = (w_crop // 2, h_crop // 2)

                # Dapatkan matriks rotasi
                M = cv2.getRotationMatrix2D(center_crop, angle_deg, 1.0)
                
                # Terapkan rotasi
                # cv2.INTER_CUBIC memberikan kualitas lebih baik
                # borderMode=cv2.BORDER_CONSTANT mengisi area hitam jika ada
                aligned_face_crop = cv2.warpAffine(face_crop_bgr, M, (w_crop, h_crop), 
                                                flags=cv2.INTER_CUBIC, 
                                                borderMode=cv2.BORDER_CONSTANT,
                                                borderValue=(0, 0, 0))
                
                # Gunakan wajah yang sudah di-align
                face_to_process = aligned_face_crop
            
            except Exception as e:
                # Jika gagal (misal landmark tidak terdeteksi), pakai crop asli
                face_to_process = face_crop_bgr

            # 6. Preprocessing wajah untuk model
            # Konversi crop (BGR) ke PIL (RGB)
            face_pil = Image.fromarray(cv2.cvtColor(face_crop_bgr, cv2.COLOR_BGR2RGB))
            
            # Terapkan transformasi
            face_tensor = transform(face_pil).unsqueeze(0).to(device)

            # 7. Dapatkan Embedding
            with torch.no_grad():
                emb = model(face_tensor)
            
            emb = emb.cpu().numpy().squeeze()

            # 8. Prediksi menggunakan Prototipe
            cls_idx, sim = predict_embedding(emb, prototypes, threshold=0.5)

            # 9. Dapatkan Label Nama
            if cls_idx == -1:
                label = "Unknown"
                color = (0, 0, 255) # Merah
            else:
                label = class_names[cls_idx]
                color = (0, 255, 0) # Hijau
            
            display_text = f"{label} ({sim*100:.1f}%)"

            # 10. Gambar Bounding Box dan Label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Hitung dan tampilkan FPS
    frame_count += 1
    end_time = time.time()
    elapsed = end_time - start_time
    if elapsed >= 1.0:
        fps = frame_count / elapsed
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        frame_count = 0
        start_time = time.time()

    # Tampilkan frame
    cv2.imshow('Real-time Face Recognition (Tekan q untuk keluar)', frame)

    # Tombol Keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
print("Stream dihentikan.")

Running on device: cuda
Loading dataset pelatihan untuk membangun prototipe...
Ditemukan 54 kelas: ['5221911012_Debora', '5221911025_Anggun', '5231811002_MichaelAndrewDeHaan', '5231811004_Hamdanu Fahmi Utomo', '5231811005_Akhmad Nabil Saputra', '5231811006_Daniel Granesa Kiara', '5231811007_Amalia Dwi Ramadhani', '5231811008_Sophia', '5231811009_Otniel Chresto Purwandi', '5231811010_MEYLAN ARYANI', '5231811013_KusumaRatih', '5231811014_Dian Eka Pratiwi', '5231811015_Fadilah Ratu Azzahra', '5231811016_Kesya', '5231811017_Maulana Ahmad Muhaimin', '5231811018_Sulis Septiani Putri', '5231811019_Chronika', '5231811021_NASHA SHINTA ABP', '5231811022_Lathif Ramadhan', '5231811023_Rahma Fieka Januarni', '5231811024_Maria Febronia Boa', '5231811025_Novera', '5231811026_ULFAH NAFIAH', '5231811027_Naufal', "5231811028_Ma'ruf Ndaru Sasono", '5231811029_Andini', '5231811030_Antonia Idan Huler', '5231811031_CindyKusumaningrum', '5231811033_Rama Panji Nararendra Cahaya', '5231811034_Shilsylia Putri D

C:\Users\ramap\AppData\Local\Temp\ipykernel_19404\3347148874.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Lear

Ekstraksi 2244 embedding pelatihan selesai.
Prototipe untuk 54 kelas berhasil dibuat.
Memulai stream video... Tekan 'q' untuk keluar.
Stream dihentikan.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2 # <-- IMPORT BARU
from torch.utils.data import DataLoader
from facenet_pytorch import MTCNN
from PIL import Image
import time

# --- Variabel & Path ---
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
# Ganti dengan path model classifier Anda
model_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\best_mobilenetv2_face.pth" 

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

# ==============================================================================
# TAHAP 1: MEMUAT MODEL CLASSIFIER DAN NAMA KELAS
# ==============================================================================
print("Loading class names from dataset directory...")
# Kita perlu dataset HANYA untuk mendapatkan nama kelas
temp_dataset = datasets.ImageFolder(root=train_dir)
class_names = temp_dataset.classes
num_classes = len(class_names)
del temp_dataset # Hapus, tidak dipakai lagi

print(f"Ditemukan {num_classes} kelas: {class_names}")

# --- Load model pre-trained (Sesuai permintaan Anda) ---
model = mobilenet_v2(weights='IMAGENET1K_V1')
num_features = model.classifier[1].in_features

# --- Ganti classifier terakhir ---
model.classifier[1] = nn.Linear(num_features, num_classes)
model = model.to(device)

# --- Load state dictionary Anda ---
print(f"Loading model weights from: {model_path}")
model.load_state_dict(torch.load(model_path, weights_only=True))
model.eval()
print("Model classifier berhasil dimuat.")

# ==============================================================================
# TAHAP 2: TRANSFORMASI
# ==============================================================================
# Transformasi untuk model (sama seperti sebelumnya)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ==============================================================================
# TAHAP 3: FUNGSI HELPER (Hanya Pencerah)
# ==============================================================================

# --- Fungsi Pencerah Gambar ---
def auto_adjust_brightness(frame, target_brightness=90, low_threshold=70):
    """Mencerahkan frame jika brightness rata-rata di bawah ambang batas."""
    
    # Konversi ke Grayscale untuk mengecek brightness
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mean_brightness = np.mean(gray)
    
    if mean_brightness < low_threshold:
        # Hitung seberapa banyak brightness perlu ditambah
        delta = target_brightness - mean_brightness
        
        # Tambah brightness menggunakan cv2.convertScaleAbs
        bright_frame = cv2.convertScaleAbs(frame, alpha=1.0, beta=delta)
        return bright_frame
        
    return frame

# ==============================================================================
# TAHAP 4: REAL-TIME LOOP
# ==============================================================================

# --- Inisialisasi MTCNN ---
mtcnn = MTCNN(
    keep_all=True,              # Deteksi semua wajah
    post_process=False,         # Jangan lakukan normalisasi
    min_face_size=40,           # Abaikan wajah yang sangat kecil
    device=device
)

# --- Inisialisasi Video Capture ---
cap = cv2.VideoCapture(0) # Gunakan webcam internal

if not cap.isOpened():
    print(f"Error: Tidak bisa membuka webcam (index 0).")
    exit()

print("Memulai stream video... Tekan 'q' untuk keluar.")

# Variabel untuk FPS
frame_count = 0
start_time = time.time()
# Tentukan threshold kepercayaan (confidence)
confidence_threshold = 0.2 # 75% - Sesuaikan jika perlu

while True:
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame.")
        break

    # 1. Pencerahan Otomatis (Sesuai permintaan Anda)
    frame_bright = auto_adjust_brightness(frame)

    # 2. Konversi BGR (OpenCV) ke RGB (PIL) untuk MTCNN
    img_pil = Image.fromarray(cv2.cvtColor(frame_bright, cv2.COLOR_BGR2RGB))

    # 3. Deteksi Wajah dengan MTCNN (dan dapatkan landmarks)
    boxes, _, landmarks = mtcnn.detect(img_pil, landmarks=True)

    # 4. Loop untuk setiap wajah yang terdeteksi
    if boxes is not None:
        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = [int(b) for b in box]

            # Sanitasi koordinat
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame.shape[1] - 1, x2)
            y2 = min(frame.shape[0] - 1, y2)

            if y2 <= y1 or x2 <= x1:
                continue

            # 5. Crop wajah dari frame BGR asli
            face_crop_bgr = frame[y1:y2, x1:x2]

            # 5.5 ALIGN Wajah menggunakan Landmark Mata
            try:
                current_landmarks = landmarks[i]
                left_eye = current_landmarks[0]
                right_eye = current_landmarks[1]
                
                delta_y = right_eye[1] - left_eye[1]
                delta_x = right_eye[0] - left_eye[0]
                angle_deg = np.degrees(np.arctan2(delta_y, delta_x))

                (h_crop, w_crop) = face_crop_bgr.shape[:2]
                center_crop = (w_crop // 2, h_crop // 2)

                M = cv2.getRotationMatrix2D(center_crop, angle_deg, 1.0)
                
                aligned_face_crop = cv2.warpAffine(face_crop_bgr, M, (w_crop, h_crop), 
                                                   flags=cv2.INTER_CUBIC, 
                                                   borderMode=cv2.BORDER_CONSTANT)
                
                face_to_process = aligned_face_crop
            
            except Exception as e:
                # Jika gagal align, pakai crop asli
                face_to_process = face_crop_bgr
            
            # 6. Preprocessing wajah untuk model
            face_pil = Image.fromarray(cv2.cvtColor(face_to_process, cv2.COLOR_BGR2RGB))
            face_tensor = transform(face_pil).unsqueeze(0).to(device)

            # --- ▼▼▼ LOGIKA PREDIKSI BARU ▼▼▼ ---
            # 7. Dapatkan Prediksi (Logits)
            with torch.no_grad():
                outputs = model(face_tensor) # Output adalah [1, num_classes]
                
                # 8. Ubah Logits ke Probabilitas (Softmax)
                probabilities = F.softmax(outputs, dim=1)
                
                # 9. Dapatkan kelas dan kepercayaan (confidence) tertinggi
                confidence, predicted_idx = torch.max(probabilities, 1)
                
                cls_idx = predicted_idx.item()
                conf = confidence.item()

            # 10. Dapatkan Label Nama berdasarkan threshold
            if conf < confidence_threshold:
                label = "Unknown"
                color = (0, 0, 255) # Merah
            else:
                label = class_names[cls_idx]
                color = (0, 255, 0) # Hijau
            
            display_text = f"{label} ({conf*100:.1f}%)"
            # --- ▲▲▲ AKHIR LOGIKA PREDIKSI BARU ▲▲▲ ---

            # 11. Gambar Bounding Box dan Label
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Hitung dan tampilkan FPS
    frame_count += 1
    end_time = time.time()
    elapsed = end_time - start_time
    if elapsed >= 1.0:
        fps = frame_count / elapsed
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        frame_count = 0
        start_time = time.time()

    # Tampilkan frame
    cv2.imshow('Real-time Face Recognition (Classifier) - Tekan q', frame)

    # Tombol Keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
print("Stream dihentikan.")

Running on device: cuda
Loading class names from dataset directory...
Ditemukan 54 kelas: ['5221911012_Debora', '5221911025_Anggun', '5231811002_MichaelAndrewDeHaan', '5231811004_Hamdanu Fahmi Utomo', '5231811005_Akhmad Nabil Saputra', '5231811006_Daniel Granesa Kiara', '5231811007_Amalia Dwi Ramadhani', '5231811008_Sophia', '5231811009_Otniel Chresto Purwandi', '5231811010_MEYLAN ARYANI', '5231811013_KusumaRatih', '5231811014_Dian Eka Pratiwi', '5231811015_Fadilah Ratu Azzahra', '5231811016_Kesya', '5231811017_Maulana Ahmad Muhaimin', '5231811018_Sulis Septiani Putri', '5231811019_Chronika', '5231811021_NASHA SHINTA ABP', '5231811022_Lathif Ramadhan', '5231811023_Rahma Fieka Januarni', '5231811024_Maria Febronia Boa', '5231811025_Novera', '5231811026_ULFAH NAFIAH', '5231811027_Naufal', "5231811028_Ma'ruf Ndaru Sasono", '5231811029_Andini', '5231811030_Antonia Idan Huler', '5231811031_CindyKusumaningrum', '5231811033_Rama Panji Nararendra Cahaya', '5231811034_Shilsylia Putri Devitasary

In [ ]:
from itertools import combinations

# Example: check intra-class vs inter-class similarities
prototypes_norm = {k: v/np.linalg.norm(v) for k,v in prototypes.items()}

intra_sims = []
inter_sims = []

for cls_idx, proto in prototypes_norm.items():
    emb_cls = X_train[y_train == cls_idx]
    emb_cls = emb_cls / np.linalg.norm(emb_cls, axis=1, keepdims=True)
    # intra-class
    for i,j in combinations(range(len(emb_cls)), 2):
        intra_sims.append(np.dot(emb_cls[i], emb_cls[j]))
    # inter-class
    for other_idx, other_proto in prototypes_norm.items():
        if other_idx != cls_idx:
            inter_sims.append(np.dot(proto, other_proto))

print("Mean intra-class sim:", np.mean(intra_sims))
print("Mean inter-class sim:", np.mean(inter_sims))


Mean intra-class sim: 0.9285825
Mean inter-class sim: 0.0108167315


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
import numpy as np

# --- Folder path ---
# img_folder = r"C:\Users\ramap\OneDrive\Documents\Kuliah\data\WhatsApp Image 2025_"
img_folder = r"D:\download_d\data_test"

# --- Helper function: conditional preprocessing ---
def preprocess_image(image_pil, brightness_thresh=0.5):
    """
    Convert to RGB PIL image -> possibly brighten if too dark -> return PIL image.
    """
    # Convert PIL to OpenCV (numpy array)
    img = np.array(image_pil)  # shape H,W,3, RGB
    img_cv = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # Convert to float [0,1]
    img_float = img_cv.astype(np.float32) / 255.0
    
    # Compute brightness (average luminance)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray) / 255.0
    
    # If too dark, adjust brightness & contrast
    if brightness < brightness_thresh:
        alpha = 1.2  # contrast
        beta = 0.1   # brightness
        img_float = np.clip(alpha * img_float + beta, 0, 1)
        # Optional gamma correction
        gamma = 1.2
        img_float = np.power(img_float, 1/gamma)
    
    # Convert back to uint8 and PIL
    img_float = (img_float * 255).astype(np.uint8)
    img_rgb = cv2.cvtColor(img_float, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    
    return img_pil

# Get all image files
img_paths = [os.path.join(img_folder, f) 
             for f in os.listdir(img_folder) 
             if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Process all images
for img_path in img_paths:
    # Load image
    image = Image.open(img_path).convert("RGB")
    
    # Conditional preprocessing
    image = preprocess_image(image, brightness_thresh=0.5)
    
    # Apply same preprocessing as your train/test set
    img_tensor = test_transform(image).unsqueeze(0).to(device)

    # Extract embedding
    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy()[0]  # shape (128,)

    # Predict using prototypes
    predicted_cls, sim = predict_embedding(embedding, prototypes, threshold=0.6)  # adjust threshold
    if predicted_cls == -1:
        predicted_name = "Unknown"
    else:
        predicted_name = train_dataset.classes[predicted_cls]

    print(f"Image: {img_path} | Predicted: {predicted_name} | Cosine sim: {sim:.3f}")

    # Display image with label
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_name}")
    plt.show()

NameError: name 'test_transform' is not defined

In [ ]:
cropper = FaceCropper()
cropper.process_folder(r"C:\Users\ramap\OneDrive\Documents\Kuliah\data", r"C:\Users\ramap\OneDrive\Documents\Kuliah\data")

d:\anaconda3\envs\deeplearning\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
d

In [1]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
from torch import nn
import torch.nn.functional as F
import torch
import math

class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

##################################  MobileFaceNet #############################################################
    
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)
class MobileFaceNet(Module):
    def __init__(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, 512, bias=False)
        self.bn = BatchNorm1d(512)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    def forward(self, x):
        # print("Input:", x.shape)
    
        out = self.conv1(x)
        # print("After conv1:", out.shape)
        
        out = self.conv2_dw(out)
        # print("After conv2_dw:", out.shape)

        out = self.conv_23(out)
        # print("After conv_23:", out.shape)

        out = self.conv_3(out)
        # print("After conv_3:", out.shape)
        
        out = self.conv_34(out)
        # print("After conv_34:", out.shape)

        out = self.conv_4(out)
        # print("After conv_4:", out.shape)

        out = self.conv_45(out)
        # print("After conv_45:", out.shape)

        out = self.conv_5(out)
        # print("After conv_5:", out.shape)

        out = self.conv_6_sep(out)
        # print("After conv_6_sep:", out.shape)

        out = self.conv_6_dw(out)
        # print("After conv_6_dw:", out.shape)

        out = self.conv_6_flatten(out)
        # print("After conv_6_flatten:", out.shape)

        out = self.linear(out)
        # print("After linear:", out.shape)

        out = self.bn(out)
        # print("After batchnorm:", out.shape)

        out = l2_norm(out)
        # print("After l2_norm:", out.shape)

        return out

## INI NYOBA MENAMBAH PAKE LAIN
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.2), # 20% kemungkinan gambar akan diubah jadi grayscale
    transforms.ColorJitter(brightness=(0.2, 1.3),  # Kecerahan antara 40% - 100% (tidak pernah lebih terang)
                            contrast=(0.7, 1.3),  # Sedikit variasi kontras
                            saturation=0.2, 
                            hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\MobileFace_Net", map_location=lambda storage, loc: storage))

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=512,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0005598380474533065, weight_decay=0.0005501701817196964)
# optimizer = torch.optim.SGD(model.parameters(), 
#                             lr=0.1, 
#                             momentum=0.9, 
#                             weight_decay=5e-4)
# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None
LR_MIN_EXP = -4  # 10**-4 = 0.0001
LR_MAX_EXP = -3  # 10**-3 = 0.001

# 2. Weight Decay (wd):
#    Biasanya antara 1e-5 dan 1e-3.
WD_MIN_EXP = -5  # 10**-5 = 0.00001
WD_MAX_EXP = -3  # 10**-3 = 0.001
# for i in range(20):
    # lr = np.random.uniform(10*LR_MIN_EXP, 10*LR_MAX_EXP)
    # wd = np.random.uniform(10*WD_MIN_EXP, 10*WD_MAX_EXP)
    # print(f"lr : {lr} | wd: {wd}")
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_best.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

C:\Users\ramap\AppData\Local\Temp\ipykernel_3256\1998673123.py:192: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Lear

NameError: name 'ArcFaceLoss' is not defined

In [108]:
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\MobileFace_Net", map_location=lambda storage, loc: storage))

C:\Users\ramap\AppData\Local\Temp\ipykernel_13204\2550465223.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learn

<All keys matched successfully>

In [2]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
from torch import nn
import torch
import math
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return nn.CrossEntropyLoss()(logits, labels)
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)
    
class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

##################################  MobileFaceNet #############################################################
    
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)
class MobileFaceNet(Module):
    def __init__(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, 512, bias=False)
        self.bn = BatchNorm1d(512)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    def forward(self, x):
        # print("Input:", x.shape)
    
        out = self.conv1(x)
        # print("After conv1:", out.shape)
        
        out = self.conv2_dw(out)
        # print("After conv2_dw:", out.shape)

        out = self.conv_23(out)
        # print("After conv_23:", out.shape)

        out = self.conv_3(out)
        # print("After conv_3:", out.shape)
        
        out = self.conv_34(out)
        # print("After conv_34:", out.shape)

        out = self.conv_4(out)
        # print("After conv_4:", out.shape)

        out = self.conv_45(out)
        # print("After conv_45:", out.shape)

        out = self.conv_5(out)
        # print("After conv_5:", out.shape)

        out = self.conv_6_sep(out)
        # print("After conv_6_sep:", out.shape)

        out = self.conv_6_dw(out)
        # print("After conv_6_dw:", out.shape)

        out = self.conv_6_flatten(out)
        # print("After conv_6_flatten:", out.shape)

        out = self.linear(out)
        # print("After linear:", out.shape)

        out = self.bn(out)
        # print("After batchnorm:", out.shape)

        out = l2_norm(out)
        # print("After l2_norm:", out.shape)

        return out

## INI NYOBA MENAMBAH PAKE LAIN
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.2), # 20% kemungkinan gambar akan diubah jadi grayscale
    transforms.ColorJitter(brightness=(0.2, 1.3),  # Kecerahan antara 40% - 100% (tidak pernah lebih terang)
                            contrast=(0.7, 1.3),  # Sedikit variasi kontras
                            saturation=0.2, 
                            hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\MobileFace_Net", map_location=lambda storage, loc: storage))

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=512,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0005598380474533065, weight_decay=0.0005501701817196964)
# optimizer = torch.optim.SGD(model.parameters(), 
#                             lr=0.1, 
#                             momentum=0.9, 
#                             weight_decay=5e-4)
# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None
LR_MIN_EXP = -4  # 10**-4 = 0.0001
LR_MAX_EXP = -3  # 10**-3 = 0.001

# 2. Weight Decay (wd):
#    Biasanya antara 1e-5 dan 1e-3.
WD_MIN_EXP = -5  # 10**-5 = 0.00001
WD_MAX_EXP = -3  # 10**-3 = 0.001
# for i in range(20):
    # lr = np.random.uniform(10*LR_MIN_EXP, 10*LR_MAX_EXP)
    # wd = np.random.uniform(10*WD_MIN_EXP, 10*WD_MAX_EXP)
    # print(f"lr : {lr} | wd: {wd}")
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_bestss.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

C:\Users\ramap\AppData\Local\Temp\ipykernel_3256\2466821108.py:298: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Lear

KeyboardInterrupt: 

In [2]:
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
from torch import nn
import torch
import math
class ArcFaceLoss(nn.Module):
    def __init__(self, num_classes, embedding_size, margin, scale):
        """
        ArcFace: Additive Angular Margin Loss for Deep Face Recognition
        (https://arxiv.org/pdf/1801.07698.pdf)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
            margin: m in the paper, the angular margin penalty in radians
            scale: s in the paper, feature scale
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.margin = margin
        self.scale = scale
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        cosine = self.get_cosine(embeddings) # (None, n_classes)
        mask = self.get_target_mask(labels) # (None, n_classes)
        cosine_of_target_classes = cosine[mask == 1] # (None, )
        modified_cosine_of_target_classes = self.modify_cosine_of_target_classes(
            cosine_of_target_classes
        ) # (None, )
        diff = (modified_cosine_of_target_classes - cosine_of_target_classes).unsqueeze(1) # (None,1)
        logits = cosine + (mask * diff) # (None, n_classes)
        logits = self.scale_logits(logits) # (None, n_classes)
        return nn.CrossEntropyLoss()(logits, labels)
        
    def get_cosine(self, embeddings):
        """
        Args:
            embeddings: (None, embedding_size)
        Returns:
            cosine: (None, n_classes)
        """
        cosine = F.linear(F.normalize(embeddings), F.normalize(self.W))
        return cosine
    
    def get_target_mask(self, labels):
        """
        Args:
            labels: (None,)
        Returns:
            mask: (None, n_classes)
        """
        batch_size = labels.size(0)
        onehot = torch.zeros(batch_size, self.num_classes, device=labels.device)
        onehot.scatter_(1, labels.unsqueeze(-1), 1)
        return onehot
        
    def modify_cosine_of_target_classes(self, cosine_of_target_classes):
        """
        Args:
            cosine_of_target_classes: (None,)
        Returns:
            modified_cosine_of_target_classes: (None,)
        """
        eps = 1e-6
        # theta in the paper
        angles = torch.acos(torch.clamp(cosine_of_target_classes, -1 + eps, 1 - eps))
        return torch.cos(angles + self.margin)
    
    def scale_logits(self, logits):
        """
        Args:
            logits: (None, n_classes)
        Returns:
            scaled_logits: (None, n_classes)
        """
        return logits * self.scale
    
class SoftmaxLoss(nn.Module):
    def __init__(self, num_classes, embedding_size):
        """
        Regular softmax loss (1 fc layer without bias + CrossEntropyLoss)
        Args:
            num_classes: The number of classes in your training dataset
            embedding_size: The size of the embeddings that you pass into
        """
        super().__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        
        self.W = torch.nn.Parameter(torch.Tensor(num_classes, embedding_size))
        nn.init.xavier_normal_(self.W)
        
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: (None, embedding_size)
            labels: (None,)
        Returns:
            loss: scalar
        """
        logits = F.linear(embeddings, self.W)
        return nn.CrossEntropyLoss()(logits, labels)
    
class Flatten(Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

def l2_norm(input,axis=1):
    norm = torch.norm(input,2,axis,True)
    output = torch.div(input, norm)
    return output

##################################  MobileFaceNet #############################################################
    
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)
class MobileFaceNet(Module):
    def ___init___(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, 512, bias=False)
        self.bn = BatchNorm1d(512)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    def forward(self, x):
        # print("Input:", x.shape)
    
        out = self.conv1(x)
        # print("After conv1:", out.shape)
        
        out = self.conv2_dw(out)
        # print("After conv2_dw:", out.shape)

        out = self.conv_23(out)
        # print("After conv_23:", out.shape)

        out = self.conv_3(out)
        # print("After conv_3:", out.shape)
        
        out = self.conv_34(out)
        # print("After conv_34:", out.shape)

        out = self.conv_4(out)
        # print("After conv_4:", out.shape)

        out = self.conv_45(out)
        # print("After conv_45:", out.shape)

        out = self.conv_5(out)
        # print("After conv_5:", out.shape)

        out = self.conv_6_sep(out)
        # print("After conv_6_sep:", out.shape)

        out = self.conv_6_dw(out)
        # print("After conv_6_dw:", out.shape)

        out = self.conv_6_flatten(out)
        # print("After conv_6_flatten:", out.shape)

        out = self.linear(out)
        # print("After linear:", out.shape)

        out = self.bn(out)
        # print("After batchnorm:", out.shape)

        out = l2_norm(out)
        # print("After l2_norm:", out.shape)

        return out


In [106]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input = torch.Tensor(2, 3, 112, 112).to(device)
net = MobileFaceNet(512).to(device)
x = net(input)
print(x.shape)

TypeError: MobileFaceNet.__init__() takes 1 positional argument but 2 were given

In [103]:


## INI NYOBA MENAMBAH PAKE LAIN
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import MobileNet_V2_Weights
from tqdm import tqdm

# --- Paths ---
# train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped OLD classed"
# test_dir  = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped OLD classed"
train_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Train Cropped MTCNN fix"
test_dir = r"D:\Code\Kuliah\Deep Learning\tugas_4\data\Dataset Sistem Presensi Wajah V2.0\Data Test Cropped MTCNN fix"

# --- Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Transforms ---
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.2), # 20% kemungkinan gambar akan diubah jadi grayscale
    transforms.ColorJitter(brightness=(0.2, 1.3),  # Kecerahan antara 40% - 100% (tidak pernah lebih terang)
                            contrast=(0.7, 1.3),  # Sedikit variasi kontras
                            saturation=0.2, 
                            hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05), scale=(0.9,1.1), shear=5),
    transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Datasets ---
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# --- Dataloaders ---
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# --- Model ---
model = MobileFaceNet(512).to(device)
model.load_state_dict(torch.load(r"D:\Code\Kuliah\Deep Learning\tugas_4\models\MobileFace_Net", map_location=lambda storage, loc: storage))

# --- Loss & Optimizer ---
criterion = ArcFaceLoss(
    num_classes=54,
    embedding_size=512,
    margin=0.5,   # standard margin; increases angular separation
    scale=30.0    # keep the same
).to(device)


optimizer = optim.Adam(model.parameters(), lr=0.0005598380474533065, weight_decay=0.0005501701817196964)
# optimizer = torch.optim.SGD(model.parameters(), 
#                             lr=0.1, 
#                             momentum=0.9, 
#                             weight_decay=5e-4)
# --- Training Loop ---
num_epochs = 50
early_stop_patience = 5  # stop if test loss hasn't improved for 5 epochs
best_loss = float('inf')
epochs_no_improve = 0
stopped_early = False
best_model_state = None
LR_MIN_EXP = -4  # 10**-4 = 0.0001
LR_MAX_EXP = -3  # 10**-3 = 0.001

# 2. Weight Decay (wd):
#    Biasanya antara 1e-5 dan 1e-3.
WD_MIN_EXP = -5  # 10**-5 = 0.00001
WD_MAX_EXP = -3  # 10**-3 = 0.001
# for i in range(20):
    # lr = np.random.uniform(10*LR_MIN_EXP, 10*LR_MAX_EXP)
    # wd = np.random.uniform(10*WD_MIN_EXP, 10*WD_MAX_EXP)
    # print(f"lr : {lr} | wd: {wd}")
for epoch in range(num_epochs):
    # --- Training ---
    model.train()
    train_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        embeddings = model(imgs)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # --- Validation ---
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for imgs, labels in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            loss = criterion(embeddings, labels)
            test_loss += loss.item() * imgs.size(0)

    test_loss /= len(test_loader.dataset)

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")

    # --- Early Stopping Check ---
    if test_loss < best_loss - 1e-5:  # small threshold for improvement
        best_loss = test_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= early_stop_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        stopped_early = True
        break

# Optionally, load the best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
# Save the best model after training / early stopping
save_path = r"D:\Code\Kuliah\Deep Learning\tugas_4\models\mobilenetv2_arcface_diffmargin_xuexing_moreagument_bestddd.pth"
torch.save(model.state_dict(), save_path)
print(f"✅ Best model saved to: {save_path}")

TypeError: MobileFaceNet.__init__() takes 1 positional argument but 2 were given